# Image Classification using TensorFlow v2.



This tutorial will explain the complete pipeline from loading data from several sources to predicting results. This is a tutorial to build an image classification model from scratch using TensorFlow v2. 

This tutorial will explain how to use GPU efficiently, load image data, build and train a convolution neural network... Data augmentation is  included in the model.

Make sure to change the Accelerator on the right to GPU.

The **objectives** of this tutorial will be:
* The tutorial is intended to be a first contact with the RoCoLe dataset.
* Create an input pipeline from different input sources, images and Excel file, using TensorFlow tools.
* Use a predefined model as feature extractor(ResNet101V2 & MobileNetV2) and own binary classifier.
* Diagnose deep learning model performance using learning curves.

# 1.Installs

In [ ]:
!pip install xlrd>=1.3.0
!pip install openpyxl

# 2.Imports

In [ ]:
try:
    %tensorflow_version 2.x

except Exception:
    pass

import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Flatten
import tensorflow_datasets as tfds
import tensorflow_hub as hub

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pathlib

import itertools

print('Tensorflow version : {}'.format(tf.__version__))

tf.keras.backend.clear_session()
tf.executing_eagerly()==True

# 3.Selecting Between Strategies

## 3.1.CPU or GPU detection

Depending on the hardware available, we will use different distribution strategies. For this version we will use GPU or CPU. 
- If more than one GPU is available, then you'll use the Mirrored Strategy
- If one GPU is available or if just the CPU is available, you'll use the default strategy.

In [ ]:
# Detect hardware
try:
    gpus = tf.config.experimental.list_logical_devices("GPU")
except ValueError:
    gpu = 0
    
# Select appropriate distribution strategy
if len(gpus) > 1:
    strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
    print('Running on multiple GPUs ', [gpu.name for gpu in gpus])
elif len(gpus) == 1:
    strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
    print('Running on single GPU ', gpus[0].name)
else:
    strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
    print('Running on CPU')
print("Number of accelerators: ", strategy.num_replicas_in_sync)

# 4.Global Parameters

The global BATCH_SIZE variable is the batch size per replica times the number of replicas in the distribution strategy.

In [ ]:
BATCH_SIZE = 64 * strategy.num_replicas_in_sync # Gobal batch size.

In [ ]:

IMAGE_SIZE=224
CLASS_NAMES=[]
IMG_COUNT=0
num_classes=0

MAIN_IMG_DIR = '../input/rocoleoriginal/Photos'
MAIN_ANN_DIR = '../input/rocoleoriginal/Annotations'


# 5.Loading and Preprocessing the Dataset

## 5.1 RoCoLe: A robusta coffee leaf images dataset

 **RoCoLe**

https://data.mendeley.com/datasets/c5yvn32dzg/2

There are two main directories:


*   Photos: contains 1560 coffee leaf images (.jpg)
*   Annotations:  Segmentation, Classification etc.




In [ ]:
data_dir = pathlib.Path(MAIN_IMG_DIR)
print("Directory:", data_dir)
IMG_COUNT = len(list(data_dir.glob('*.jpg')))
print("Number of images:",IMG_COUNT)

As you can see it is a small dataset composed by 1560 pictures of coffee leaf images. Beause of that, we are going  to perform a binary classification.
Also, the data is divided in several inputs : 
* Images files  
* xlsx files.

In [ ]:
#We need to use the file where classes are described:
!ls ../input/rocoleoriginal/Annotations

We cannot created a tf.data.Dataset from image files in a directory since the information is separated in two different directories and files. 

Therefore, we will create a dataset containing location and tagging information using Pandas. Then, we will convert it into a shuffle dataset (from tensor slices). Finally we will create the dataset with the processed images.

## 5.1. Explore data

We want to perform a binary classification, so we are going to drop unnecessary information. As we can see, Multiclass label shall be removed.

In [ ]:
df = pd.read_excel(MAIN_ANN_DIR+'/RoCoLe-classes.xlsx',engine='openpyxl')
df.head()

In [ ]:
df.drop(['Multiclass.Label'], axis=1, inplace=True)
df.rename(columns={'Binary.Label':'Label'},inplace=True)

Run the following cell to see how many healthy/unhealthy pictures we have .

In [ ]:
#How many Labels has Multiclass.Label? R: 6
#How many Labels has Binary.Label? R: 2
num_classes=df.drop_duplicates(subset = ["Label"]).count()["Label"]
print("Number of Labels(in Binary.Label  column): "+ str(num_classes))
df.drop_duplicates(subset = ["Label"])

check_imbalance = df.pivot_table(index=['Label'], aggfunc='size')
print (check_imbalance)


Notice that the there are a little more images that are classified as healthy than unhealthy. However, this not shows that we have big imbalance in our data, because the diference between healthy and unhealthy is only 22 images.

Let's change the Label column from string to integer. Also, we are going to create a dictionary of Words, we will use this latter.

In [ ]:
CLASS_NAMES = (pd.Series.to_string(df.drop_duplicates(subset = ["Label"])["Label"],index=False).strip()).split()
CLASS_NAMES.sort()
print(CLASS_NAMES)

In [ ]:
dictOfWords = { CLASS_NAMES[i]:i for i in range(0, len(CLASS_NAMES))}
dictOfWords

In [ ]:
df["Label"] = df["Label"].map(dictOfWords)
df

## 5.2. Process Data

### 5.2.1.Load Pandas Dataframe as ShuffleDataset

Define 2 dataset:

* train_ds = the training set, 80%.
* val_ds    = the validation set, 20%.

Test dataset is not available because we are going to use the 20% percent to create the validation dataset.


In [ ]:
target = df.pop('Label')
dir_img=MAIN_IMG_DIR+'/{}'
df=df.applymap(dir_img.format)


dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))
dataset = dataset.shuffle(IMG_COUNT, reshuffle_each_iteration=False)

val_size = int(IMG_COUNT * 0.20)
train_ds = dataset.skip(val_size)
val_ds = dataset.take(int(val_size))


train_size=tf.data.experimental.cardinality(train_ds).numpy()
val_size=tf.data.experimental.cardinality(val_ds).numpy()

print("Training size: {}".format(train_size))
print("Validation size: {}".format(val_size))
print("Total images: ",train_size+val_size)
print("\nExample: ")
for feat, targ in train_ds.take(5):
    print ('Image: {}, Label: {}'.format(feat, targ))

### 5.2.2.Configure dataset and Performance
Define some helper functions that will pre-process our data:

* parse_image: load,decode,convert...an image.
* get_training_dataset: loads data and splits it to get the training set.
* get_validation_dataset: loads and splits the data to get the validation set.

In [ ]:
'''
Transforms each image in dataset 
'''

def parse_image(feat, targ):

    image = tf.io.read_file(feat[0])
    image = tf.image.decode_jpeg(image)
    image = tf.image.convert_image_dtype(image, tf.float32)

    # image pretreatment 
    image = tf.image.resize(image, [1512, 1512])
    image =  tf.image.central_crop(image, central_fraction=0.65)
    image = tf.image.resize(image, [IMAGE_SIZE, IMAGE_SIZE]) 
  
    label= tf.cast(targ,tf.int32)

    return image, label
'''
Loads and maps the training split of the dataset using the map function. 
'''
AUTOTUNE = tf.data.experimental.AUTOTUNE
def get_training_dataset():

      with  strategy.scope():
        dataset = train_ds.map(parse_image, num_parallel_calls=16)
        dataset = dataset.shuffle(buffer_size=train_size,
                                  reshuffle_each_iteration=True)
        dataset = dataset.repeat() 
        dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
        dataset = dataset.prefetch(AUTOTUNE) 
        return dataset

'''
Loads and maps the validation split of the dataset using the map function. 
'''  
def get_validation_dataset():

    dataset = val_ds.map(parse_image, num_parallel_calls=16)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True) 
    dataset = dataset.shuffle(buffer_size=train_size, 
                              reshuffle_each_iteration=True)
    dataset = dataset.repeat()
    dataset = dataset.prefetch(AUTOTUNE) 
    return dataset

# instantiate the datasets
with strategy.scope():
    training_dataset = get_training_dataset()
    validation_dataset = get_validation_dataset()

### 5.2.3.Visualize

In [ ]:
image_batch, label_batch = next(iter(training_dataset))

plt.figure(figsize=(10, 10))
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image_batch[i])
    label = label_batch[i]
    #index = tf.argmax(label.numpy(), axis=0)
    plt.title(CLASS_NAMES[label])


# 6.Models

We will use two predefined models, which we will use the weights provided by imagenet. 

## 6.1.Utilities and model variables

In [ ]:
#Training variables
EPOCHS = 100
steps_per_epoch = (train_size)//BATCH_SIZE 
validation_steps = (val_size)//BATCH_SIZE 
IMG_DIM=(IMAGE_SIZE,IMAGE_SIZE)

In [ ]:
#Callback functions
early_stopping = tf.keras.callbacks.EarlyStopping(patience=20,
                                                  restore_best_weights=True)

def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 **(epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(0.01, 20)
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(exponential_decay_fn)

In [ ]:
# Plotting Training and Validation Accuracy
def plot_history(hist,Text):
  history=hist
  acc = history.history["accuracy"]
  val_acc = history.history["val_accuracy"]

  loss = history.history["loss"]
  val_loss = history.history["val_loss"]

  epochs_range = range(len(history.epoch))

  plt.figure(figsize=(10,10))
  plt.subplot(1,2,1)
  plt.plot(epochs_range,acc,label="Train Accuracy")
  plt.plot(epochs_range,val_acc,label="Validation Accuracy")
  plt.legend(loc = 'lower right')
  plt.title("Accuracy")
  plt.subplot(1,2,2)
  plt.plot(epochs_range,loss,label="Train Loss")
  plt.plot(epochs_range,val_loss,label="Validation Loss")
  plt.legend(loc = 'upper right')
  plt.title("Loss")
  plt.suptitle(Text)

## 6.2.ResNet101V2

Our model has the following sequential structure:

Data augmentation -> Feature extractor (resnet) -> Binary clasification

### 6.2.1.Define model

In [ ]:
IMG_DIM=(IMAGE_SIZE,IMAGE_SIZE)
def make_model():
    data_augmentation = tf.keras.Sequential([
    tf.keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),  # 128x128x3
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(IMAGE_SIZE, 
                                                              IMAGE_SIZE,
                                                              3)),
     tf.keras.layers.experimental.preprocessing.RandomFlip("vertical", 
                                                 input_shape=(IMAGE_SIZE, 
                                                              IMAGE_SIZE,
                                                              3)),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.5),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.80)
    ])
    
    base_model = tf.keras.applications.ResNet101V2(input_shape=(*IMG_DIM, 3),
                                             include_top=False,
                                             weights= 'imagenet')
    base_model.trainable = False
    
    model = tf.keras.Sequential([
        data_augmentation, 
        base_model,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1024, activation="relu"),
         tf.keras.layers.Dropout(0.4),
         tf.keras.layers.Dense(512, activation="relu" ),
         tf.keras.layers.Dropout(0.4),
         tf.keras.layers.BatchNormalization(),
         tf.keras.layers.Dense(num_classes-1,activation="sigmoid")
])
        
    return model

### 6.2.2.Compile model

In [ ]:
with strategy.scope():
    model = make_model()
    model.compile(loss="binary_crossentropy",
    optimizer='adam',
    metrics=["accuracy"])

In [ ]:
model.summary()

### 6.2.3.Train model

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint("RN101v2.h5",
                                                save_best_only=True)
history1 = model.fit(training_dataset,steps_per_epoch=steps_per_epoch, epochs=EPOCHS, 
                    validation_data = validation_dataset, validation_steps=validation_steps, 
                    batch_size=BATCH_SIZE, 
                     callbacks=[checkpoint, early_stopping,lr_scheduler])

### 6.2.4.Plot results

In [ ]:
# Plotting Training and Validation Accuracy
plot_history(history1,"ResNet101V2")


## 6.3.MobileNetV2

Our model has the following sequential structure:

Data augmentation -> Feature extractor (MobileNetV2) -> Binary clasification

### 6.3.1.Define model

In [ ]:

def make_model():
    data_augmentation = tf.keras.Sequential([
    tf.keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),  # 128x128x3
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(IMAGE_SIZE, 
                                                              IMAGE_SIZE,
                                                              3)),
     tf.keras.layers.experimental.preprocessing.RandomFlip("vertical", 
                                                 input_shape=(IMAGE_SIZE, 
                                                              IMAGE_SIZE,
                                                              3)),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.5),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.80)
    ])
    
    base_model = tf.keras.applications.MobileNetV2(input_shape=(*IMG_DIM, 3),
                                             include_top=False,
                                             weights= 'imagenet')
    base_model.trainable = False
    
    model = tf.keras.Sequential([
        data_augmentation, 
        base_model,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1024, activation="relu"),
         tf.keras.layers.Dropout(0.4),
         tf.keras.layers.Dense(512, activation="relu" ),
         tf.keras.layers.Dropout(0.4),
         tf.keras.layers.BatchNormalization(),
         tf.keras.layers.Dense(num_classes-1,activation="sigmoid")
])
        
    return model

### 6.3.2.Compile model

In [ ]:
with strategy.scope():
    model2 = make_model()
    model2.compile(loss="binary_crossentropy",
    optimizer='adam',
    metrics=["accuracy"])

In [ ]:
model2.summary()

### 6.3.3.Train model

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint("MNetV2.h5",
                                                save_best_only=True)
history2 = model2.fit(training_dataset,steps_per_epoch=steps_per_epoch, epochs=EPOCHS, 
                    validation_data = validation_dataset, validation_steps=validation_steps, 
                    batch_size=BATCH_SIZE, 
                     callbacks=[checkpoint, early_stopping,lr_scheduler])

### 6.3.4.Plot results

In [ ]:
# Plotting Training and Validation Accuracy
plot_history(history2,"MobileNetV2")

# 7.Evaluation and conclusions


"*When the learning curve for training loss that shows improvement and the learning curve for validation loss also shows improvement, but a large gap remains between both curves, we can conclude that we have an unrepresentative training dataset.* 

...

*Also,when the learning curve for training looks that achieves a good fit and the learning curve for validation loss shows noisy movements around the training loss or the validation loss is inferior than training loss, we can conclude that we have an unrepresentative validation dataset*". 

...

*This may occur if the training dataset has too few examples as compared to the validation dataset* (or viceversa).

...

*Unrepresentative validation dataset implies that the ability of the model to generalize is very poor because the validation dataset does not provide enough information. While unrepresentative training dataset means that the training dataset does not provide sufficient information to learn the problem, relative to the validation dataset used to evaluate it.*"

Source [Jason Brownlee ](https://machinelearningmastery.com/learning-curves-for-diagnosing-machine-learning-model-performance/)

Although RoCoLe has information to be able to classify in binary it is not enough for these two trained networks. In the article they explain that they have used other types of ML algorithms such as SVM. However, they don’t give more information nor results.

Anyway, the tutorial has been satisfactory to show all the essential steps to start analyzing a dataset using TensorFlow tools. It has been demonstrated, using visual tools, that this dataset is not optimal for a binary classification following the above steps.

Hope you liked it!